# Bioinformcatics tutorial (ver. 1)
## 2020/05/21 Minato Yamashita

Welcome to this tutorial!  
Here, we analyze single cell RNA-seq data with a powerful Python library called Scanpy.  
I hope this tutorial helps you be familiar with and enjoy bioinformatics!  

### Step 1: read data
First of all, load necessary libraries and read data you downloaded.

In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import scanpy as sc

In [ ]:
sc.settings.verbosity = 3 # set the parameter to 3 to see hints

You can read h5ad file by executing ` read_h5ad` function.

In [ ]:
adata = sc.read_h5ad('pellin_2019.h5ad')

Let's check the structure of the data.

In [ ]:
adata

### Step 2: understand the data structure
Let's see how the data is organized.

In [ ]:
adata.var # this shows the data related to genes

In [ ]:
adata.obs # this shows the data related to cells

### Step 3: investigate genes

Now, you can manipulate the data freely.  
Let's explore which genes express highly and what functions they have.  
Plus, check the proportions of zero counts.

In [ ]:
sc.pl.highest_expr_genes(adata, n_top=20)

Check what functions genes you are interested in have on [Ensembl Genome Browser](https://asia.ensembl.org/index.html).

Or, are you unhappy with the result above?  
If so, you can filter out some genes like the following.

In [ ]:
sc.pl.highest_expr_genes(adata[:, ~adata.var.index.isin(['HBB', 'HBA1', 'HBA2'])], n_top=20)

Next, let's calculate dropout rates.

In [ ]:
def dropout(adata):
    dropout_rate = np.count_nonzero(adata.X==0, axis=1) / adata.n_vars
    return dropout_rate

In [ ]:
dropout_rate = dropout(adata)

In [ ]:
sns.distplot(dropout_rate)

### Step 4: pre-process the data

You have got familiar with the data you analyze, haven't you?  
OK, let's move on to a pre-processing phase.

First, you need filter out low-quality cells and genes.  
I set the thresholds to common, widely used values.  
But, these are arbitrary, and you can use other values you think are better.

In [ ]:
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

In addition, calculate the proportions of counts of mitochodrial genes, and filter out cells which have higher proportions of mitchondrial genes.

In [ ]:
adata.var['mt'] = adata.var_names.str.startswith('MT.')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

Now, you can plot:
* n_genes_by_counts: the number of expressed genes in each cell
* total_counts: the number of total counts in each cell
* pct_count_mt: the percentages of mitochondrial genes in each cell

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'], jitter=0.4, multi_panel=True)

In [ ]:
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')

It is common to remove cells which have too many counts and / or high proportion of mitochondrial gens.  
Why do you need to do so?  
Any thoughts?

Again, these thresholds are arbitrary.  
Any other values are better?

In [ ]:
adata = adata[adata.obs.n_genes_by_counts < 2500, :]
adata = adata[adata.obs.pct_counts_mt < 5, :]

Normalize and logarithmize the data.  
Each cell has the different number of total counts, so it is common to normalize the data by making each cell have the same number of total counts.  
Here, we normalize the data so that each cell has 1 million counts in total.  
By doing so, counts of each cell are transformed to CPM (count per million).

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e6)
sc.pp.log1p(adata)

There are too many genes in the data, and most of them are not so important and noisy.  
It is critical to extract important genes to make results less noisy and reduce calculation time.

Here, we calculate 

In [ ]:
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

dispersion = variance / mean  
By setting thresholds to min_mean, max_mean and min_disp, you can extract highly variable genes across cells which can extract differences among cells.

In [ ]:
sc.pl.highly_variable_genes(adata)

As you guess, these thresholds are arbitrary and a little bit rough.  
This step is important for later analyses, so you can return here after executing them.

In [ ]:
# save the data for later use
adata.raw = adata

Filter the data with extracted highly variable genes.  
Regress out effects of unwanted variances of total counts and percentages of mitochondrial genes.  
Set the highest standard deviation, and make cells have a unit variance.

In [ ]:
adata = adata[:, adata.var.highly_variable]
sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])
sc.pp.scale(adata, max_value=10)

## Step 5: principal component analysis
Let's reduce the dimensions of the data and visualize it!

In [ ]:
sc.tl.pca(adata, svd_solver='arpack')

In [ ]:
sc.pl.pca(adata, color=adata.obs)

In [ ]:
sc.pl.pca(adata, color='cell_markers')

Let's investigate the result.  
Each principal component (= PC) explains the fraction of total variance of the original data.  
And, the order of PCs (= PC1, PC2, ...) depends on what proportions of the total variance they explain.  
Plot the explained variances by each PC (this plot is called *scree plot*)

In [ ]:
sc.pl.pca_variance_ratio(adata, log=True, n_pcs=50)

You notice an "elbow" around PC7.  
This means that critical information is stored in PC1-PC7, and PC8-PC50 contain not so important information and may just make later calculation more complex.  
So, you can set a thershold of PC you use to PC7.  

## Step 6: compute the neighborhood graph
To extract important relationships between cells, let's make k-nearest neighbor graph based on the result of PCA.

In [ ]:
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=7) # try to change the parameters!

## Step 7: embed the graph
Now, you can visualize your data on 2D space with more sophisticated methods, that is t-SNE and UMAP.  
Here, you try the two methods and campare the results.

In [ ]:
sc.tl.umap(adata)

In [ ]:
sc.tl.tsne(adata, n_pcs=7)

In [ ]:
sc.pl.umap(adata, color='cell_markers')
sc.pl.tsne(adata, color='cell_markers')

## Step 8: cluster the graph

In [ ]:
sc.tl.leiden(adata)

In [ ]:
sc.tl.louvain(adata)

In [ ]:
sc.pl.umap(adata, color='louvain')
sc.pl.tsne(adata, color='louvain')
sc.pl.umap(adata, color='leiden')
sc.pl.tsne(adata, color='leiden')

## Step 9: find marker genes of clusters
Next, let's find marker genes and difne cell types.  
There are several ways to find differentially expressed genes between clusters.  
Here, we use one of the most basic method.

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden', method='wilcoxon')
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

Construct a data frame to store marker genes.

In [ ]:
marker_df = pd.DataFrame(adata.uns['rank_genes_groups']['names'])
marker_df.head(10)

To check the clusters, let's call back the previous UMAP plot.

In [ ]:
sc.pl.umap(adata, color='leiden')

According to [the original paper](https://www.nature.com/articles/s41467-019-10291-0#Sec10), the authors used the following marker genes to identify cell types:  
(cell type, abbreviation, marker gene) =  
(early progenitor cells, P, [HLF, ADGRG6, CRHBP, PCDH9]),  
(megakaryocytes, Meg, [ITGA2B, PLEK]),  
(erythroid cells, E, [KLF1, CA1]),  
(basophil progenitors, BaP, [CLC, CPA3, HDC]),  
(neutrophils, N, [ELANE]),  
(monocytes, M, [LYZ, MS4A6A, ANXA2]),  
(dendritic cells, DC, [IRF8, SPIB, IGKC]),  
(lymphoid T/B/NK cells, Ly, [DNTT, CD79A, VPREB1])

Let's make a lost of marker genes for later use.

In [ ]:
marker_genes = ['HLF', 'ADGRG6', 'CRHBP', 'PCDH9', 'ITGA2B', 'PLEK', 'KLF1', 'CA1', 
                         'CLC', 'CPA3', 'HDC', 'ELANE', 'LYZ', 'MS4A6A', 'ANXA2', 
                         'IRF8', 'SPIB', 'IGKC', 'DNTT', 'CD79A', 'VPREB1']

Next, let's annotate each cluster with marker genes of each cell type.

In [ ]:
def find_markers(df, marker_genes):
    for i in range(len(df.columns)):
        print(f'Cluster {i} contains', df.iloc[:, i][df.iloc[:, i].isin(marker_genes)].values)

In [ ]:
find_markers(marker_df, marker_genes)

In [ ]:
cell_types = ['N/M', 'E', 'unknown', 'E', 'E', 'unknown', 'E', 'E', 'unknown', 'unknown', 'E', 'E', 'Ly', 'BaP']

Using these markers, annotate clusters.

In [ ]:
leiden = [str(i) for i in range(14)]
leiden_to_cell = dict(zip(leiden, cell_types))
adata.obs['cell_types'] = (adata.obs['leiden'].map(leiden_to_cell).astype('category'))

OK!  
Let's visualize the UMAP embedding with the new colors (= cell types)!

In [ ]:
sc.pl.umap(adata, color='cell_types', title='', frameon=False)

Finally, let's check the gene expressions of marker genes among cell types.

In [ ]:
sc.pl.dotplot(adata, marker_genes, groupby='cell_types');

In [ ]:
sc.pl.stacked_violin(adata, marker_genes, groupby='cell_types', rotation=90);

If you want to save the result, run the code below.

In [ ]:
adata.write('result.h5ad')